In [1]:
import os
import geopandas as gpd
from sentinelsat import SentinelAPI, make_path_filter
from IPython import display # lets us display images that we download
import shapely
import requests
import netrc
import pystac_client
import stackstac
import matplotlib.pyplot as plt

In [2]:
point_lat, point_lon = 45.5, -95.5 # Kolstad Lake Waterfowl Production Area
sentinel_search_url = "https://earth-search.aws.element84.com/v1"
sentinel_stac_client = pystac_client.Client.open(sentinel_search_url)
items = sentinel_stac_client.search(
       intersects=dict(type="Point", coordinates=(point_lon, point_lat)),
       collections=["sentinel-2-l2a"]).item_collection()
len(items)
1518

1518

In [3]:
sentinel_stack = stackstac.stack(items, assets=["red", "nir", "scl"],
                          bounds=[point_lon-0.0005, point_lat-0.0005, point_lon+0.0005, point_lat+0.0005],
                          gdal_env=stackstac.DEFAULT_GDAL_ENV.updated(
                               {'GDAL_HTTP_MAX_RETRY': 3,
                                'GDAL_HTTP_RETRY_DELAY': 5,
                               }),
                          epsg=4326, chunksize=(1, 1, 50, 50)).rename(
       {'x': 'lon', 'y': 'lat'}).to_dataset(dim='band')

C:\ProgramData\anaconda3\envs\EGM722\Lib\site-packages\stackstac\prepare.py:408: UserWarning: The argument 'infer_datetime_format' is deprecated and will be removed in a future version. A strict version of it is now the default, see https://pandas.pydata.org/pdeps/0004-consistent-to-datetime-parsing.html. You can safely remove this argument.
  times = pd.to_datetime(


In [4]:
sentinel_stack

<xarray.Dataset> Size: 11MB
Dimensions:                                  (time: 2401, lat: 12, lon: 8,
                                              band: 3)
Coordinates: (12/52)
  * time                                     (time) datetime64[ns] 19kB 2017-...
    id                                       (time) <U24 230kB 'S2A_15TUL_201...
  * lon                                      (lon) float64 64B -95.5 ... -95.5
  * lat                                      (lat) float64 96B 45.5 ... 45.5
    earthsearch:payload_id                   (time) <U74 711kB 'roda-sentinel...
    s2:dark_features_percentage              (time) object 19kB 0.467518 ... 0
    ...                                       ...
    gsd                                      (band) object 24B 10 10 None
    raster:bands                             (band) object 24B None ... [{'no...
    common_name                              (band) object 24B 'red' 'nir' None
    center_wavelength                        (band) object 24B 0.665 0.842 None
    full_width_half_max                      (band) object 24B 0.038 0.145 None
    epsg                                     int32 4B 4326
Dimensions without coordinates: band
Data variables:
    red                                      (time, lat, lon) float64 2MB dask.array<chunksize=(1, 12, 8), meta=np.ndarray>
    nir                                      (time, lat, lon) float64 2MB dask.array<chunksize=(1, 12, 8), meta=np.ndarray>
    scl                                      (time, lat, lon) float64 2MB dask.array<chunksize=(1, 12, 8), meta=np.ndarray>
Attributes:
    spec:           RasterSpec(epsg=4326, bounds=(-95.50052812266786, 45.4994...
    crs:            epsg:4326
    transform:      | 0.00, 0.00,-95.50|\n| 0.00,-0.00, 45.50|\n| 0.00, 0.00,...
    resolution_xy:  (0.00013326132348367992, 9.283105634949607e-05)

In [5]:
sentinel_stack['ndvi'] = (sentinel_stack['nir'] - sentinel_stack['red'])/\
                        (sentinel_stack['nir'] + sentinel_stack['red'])
sentinel_stack = sentinel_stack[['ndvi', 'scl']]
sentinel_stack = sentinel_stack.drop([c for c in sentinel_stack.coords if not (c in ['time', 'lat', 'lon'])])
sentinel_stack

C:\Users\kinse\AppData\Local\Temp\ipykernel_27728\1659960584.py:4: DeprecationWarning: dropping variables using `drop` is deprecated; use drop_vars.
  sentinel_stack = sentinel_stack.drop([c for c in sentinel_stack.coords if not (c in ['time', 'lat', 'lon'])])


<xarray.Dataset> Size: 4MB
Dimensions:  (time: 2401, lat: 12, lon: 8)
Coordinates:
  * time     (time) datetime64[ns] 19kB 2017-01-03T17:30:23.064000 ... 2024-0...
  * lon      (lon) float64 64B -95.5 -95.5 -95.5 -95.5 -95.5 -95.5 -95.5 -95.5
  * lat      (lat) float64 96B 45.5 45.5 45.5 45.5 45.5 ... 45.5 45.5 45.5 45.5
Data variables:
    ndvi     (time, lat, lon) float64 2MB dask.array<chunksize=(1, 12, 8), meta=np.ndarray>
    scl      (time, lat, lon) float64 2MB dask.array<chunksize=(1, 12, 8), meta=np.ndarray>
Attributes:
    spec:           RasterSpec(epsg=4326, bounds=(-95.50052812266786, 45.4994...
    crs:            epsg:4326
    transform:      | 0.00, 0.00,-95.50|\n| 0.00,-0.00, 45.50|\n| 0.00, 0.00,...
    resolution_xy:  (0.00013326132348367992, 9.283105634949607e-05)

In [6]:
sentinel_point = sentinel_stack.interp(lat=point_lat, lon=point_lon,
                                      method="nearest")
sentinel_stack

<xarray.Dataset> Size: 4MB
Dimensions:  (time: 2401, lat: 12, lon: 8)
Coordinates:
  * time     (time) datetime64[ns] 19kB 2017-01-03T17:30:23.064000 ... 2024-0...
  * lon      (lon) float64 64B -95.5 -95.5 -95.5 -95.5 -95.5 -95.5 -95.5 -95.5
  * lat      (lat) float64 96B 45.5 45.5 45.5 45.5 45.5 ... 45.5 45.5 45.5 45.5
Data variables:
    ndvi     (time, lat, lon) float64 2MB dask.array<chunksize=(1, 12, 8), meta=np.ndarray>
    scl      (time, lat, lon) float64 2MB dask.array<chunksize=(1, 12, 8), meta=np.ndarray>
Attributes:
    spec:           RasterSpec(epsg=4326, bounds=(-95.50052812266786, 45.4994...
    crs:            epsg:4326
    transform:      | 0.00, 0.00,-95.50|\n| 0.00,-0.00, 45.50|\n| 0.00, 0.00,...
    resolution_xy:  (0.00013326132348367992, 9.283105634949607e-05)

In [9]:
sentinel_point.load()

RuntimeError: Error reading Window(col_off=0, row_off=0, width=8, height=12) from 'https://sentinel-cogs.s3.us-west-2.amazonaws.com/sentinel-s2-l2a-cogs/14/T/QR/2020/4/S2B_14TQR_20200422_1_L2A/B04.tif': RasterioIOError('Read or write failed. IReadBlock failed at X offset 0, Y offset 0: /vsicurl/https://sentinel-cogs.s3.us-west-2.amazonaws.com/sentinel-s2-l2a-cogs/14/T/QR/2020/4/S2B_14TQR_20200422_1_L2A/B04.tif, band 1: IReadBlock failed at X offset 7, Y offset 5: TIFFReadEncodedTile() failed.')

In [ ]:
sentinel_table = sentinel_point.to_dataframe()

In [ ]:
sentinel_table['ndvi'].plot(label='unfiltered', marker='o', linestyle='',markersize=2)

In [ ]:
sentinel_table_filtered = sentinel_table[(sentinel_table['scl'] == 4) |
                                        (sentinel_table['scl'] == 5)]
sentinel_table_filtered['ndvi'].plot(label='unfiltered', marker='o', linestyle='',markersize=2)